# sqlite3으로 데이터베이스 생성하기

In [1]:
import sqlite3


def create_sqlite_database(filename):
    conn = None
    try:
        conn = sqlite3.connect(filename)
        print(sqlite3.sqlite_version)
    except sqlite3.Error as e:
        print(type(e))
        print('Err:', e)
    finally:
        if conn:
            conn.close()

다음을 실행하면 실행 위치에 'my.db'라는 데이터베이스 파일이 생성된다.

In [2]:
!ls

EDA_mtcars.ipynb       northwind.sql          sql_tutorial.ipynb
mtcars.csv             northwind_bak.db       sqlite3_tutorial.ipynb
northwind.db           northwind_original.db


In [3]:
create_sqlite_database('my.db')

3.43.2


In [4]:
!ls

EDA_mtcars.ipynb       northwind.db           northwind_original.db
mtcars.csv             northwind.sql          sql_tutorial.ipynb
my.db                  northwind_bak.db       sqlite3_tutorial.ipynb


만약 데이터베이스 파일을 생성할 수 없거나 불러올 수 없다면 sqlite3.Error의 파생 오류가 발생하게 된다.

In [5]:
!chmod -r my.db

In [6]:
create_sqlite_database('my.db')

<class 'sqlite3.OperationalError'>
Err: unable to open database file


In [7]:
!rm my.db

파일 이름을 ':memory:'로 지정하면 파일 생성 없이 메모리 안에서 데이터베이스가 생성된다.

In [8]:
create_sqlite_database(':memory:')

3.43.2


# SQLite 테이블 생성하기

In [9]:
sql_statements = [ 
    """CREATE TABLE IF NOT EXISTS projects (
            id INTEGER PRIMARY KEY, 
            name text NOT NULL, 
            begin_date TEXT, 
            end_date TEXT
    );""",
    """CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY, 
            name TEXT NOT NULL, 
            priority INT, 
            project_id INT NOT NULL, 
            status_id INT NOT NULL, 
            begin_date TEXT NOT NULL, 
            end_date TEXT NOT NULL, 
            FOREIGN KEY (project_id) REFERENCES projects (id)
    );""",
    ]

# create a database connection
try:
    with sqlite3.connect('my.db') as conn:
        cursor = conn.cursor()
        for statement in sql_statements:
            cursor.execute(statement)

        conn.commit()
except sqlite3.Error as e:
    print(e)

# SQLite에서 데이터베이스 파일 생성하지 않고 불러오기

- URI로 불러오기
    - 장점
        - 모드로 접근방식을 결정해 읽기 전용으로 불러올 수 있음
        - 파일이 없다면 `sqlite3.OperationalError`을 발생시키기 때문에 에러 핸들링 가능
        - URI로 불러오는 작업이 atomic 하기에 파일을 확인하고 불러오는 사이에 파일이 삭제되거나 사용중일 가능성 제거
    - 단점
        - URI 구조를 알아야 함
        - `uri=True`를 추가해야하는데, 이를 까먹으면 의도대로 작동하지 않을 수 있음
        - 시스템이나 SQLite 버전에서 지원하지 않는다면 사용할 수 없음
        - `SQLITE_OPEN_EXCLUSIVE`와 같은 플래그를 사용할 수 없음
        - 데이터베이스 파일을 불러오는데 실패했는데 파일이 없어서인지 권한이 없어서인지 알 수 없음

In [10]:
try:
    conn = sqlite3.connect('file:my.db?mode=rw', uri=True)
    print('Database loaded successfully.')
except sqlite3.Error as e:
    print(type(e))
    print(e)
finally:
    conn.close()

Database loaded successfully.


- `os.path.exists`로 확인하기
    - 장점
        - 파일 체크를 명시적으로 적다보니 눈에 잘 띤다
        - 어떤 시스템에서든 사용할 수 있다
        - SQLite 플래그도 사용할 수 있다
        - 파일이 없는 경우로 분기를 추가할 수 있다
    - 단점
        - 에러 핸들링과 별개로 코드를 작성해야 함
        - 파일 체크와 데이터베이스 불러오는 작업 중간에 파일이 삭제되거나 다른 프로세스가 쓸 수 있다

In [11]:
import os


db_path = 'my.db'
if os.path.exists(db_path):
    try:
        conn = sqlite3.connect(db_path)
        print('Database loaded successfully')
    except sqlite3.Error as E:
        print('Err:', e)
else:
    print(f'Err: {db_path} doesn\'t exist')

Database loaded successfully


In [12]:
import string

def char_generator():
    for c in string.ascii_lowercase:
        yield (c,)

con = sqlite3.connect(":memory:")
cur = con.cursor()
cur.execute("create table characters(c)")

cur.executemany("insert into characters(c) values (?)", char_generator())

cur.execute("select c from characters")
print(cur.fetchall())

[('a',), ('b',), ('c',), ('d',), ('e',), ('f',), ('g',), ('h',), ('i',), ('j',), ('k',), ('l',), ('m',), ('n',), ('o',), ('p',), ('q',), ('r',), ('s',), ('t',), ('u',), ('v',), ('w',), ('x',), ('y',), ('z',)]
